In [7]:
from keras.models import load_model
import os
import pandas as pd
from PIL import Image
from numpy import asarray
import numpy as np
from scipy.spatial.distance import cosine
from mtcnn.mtcnn import MTCNN
from keras_vggface.utils import preprocess_input
from matplotlib import pyplot
import pickle
import matplotlib.pyplot as plt

In [8]:
# extract a single face from a given photograph
def extract_face(filename, required_size=(224, 224)):
    # load image from file
    pixels = pyplot.imread(filename)
    # create the detector, using default weights
    detector = MTCNN()
    # detect faces in the image
    results = detector.detect_faces(pixels)
    # extract the bounding box from the first face
    x1, y1, width, height = results[0]['box']
    x2, y2 = x1 + width, y1 + height
    # extract the face
    face = pixels[y1:y2, x1:x2]
    # resize pixels to the model size
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = asarray(image)
    return face_array

In [11]:
def extract(filename, required_size=(224, 224)):
    # load image from file
    face = pyplot.imread(filename)
    # resize pixels to the model size
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = asarray(image)
    return face_array

In [9]:
# extract faces and calculate face embeddings for a list of photo files
iden = []
def get_embeddings(filenames):
    # extract faces
    array = []
    for f in filenames:
        iden.append(f)
        try:
            faces = extract_face('./dataset/' + f)
        except:
            print('Face not detected taking whole image {}'.format(f))
            try:
                faces = pyplot.imread('./dataset/' + f)
                faces = faces.resize((224, 224))
                faces = asarray(image)
            except:
                print("Whole image couldn't be taken.{} is skipped".format(f))
                continue
        # convert into an array of samples
        samples = asarray(faces, 'float32')
        # prepare the face for the model, e.g. center pixels
        samples = preprocess_input(samples, version=2)
        # perform prediction
        samples = samples.reshape(1,224,224,3)
        yhat = model.predict(samples)
        array.append(yhat)
    return array

In [4]:
# define filenames
filenames = os.listdir('./dataset/')
iden.clear()

model = load_model('face_recog.h5')
embeddings = get_embeddings(filenames)

with open("embeddings.txt", "wb") as fp:   #Pickling
    pickle.dump(embeddings, fp)
    
with open("identity.txt", "wb") as fp:   #Pickling
    pickle.dump(iden, fp)

C:\Users\anuj4\AppData\Local\conda\conda\envs\TF\lib\site-packages\keras\engine\saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Face not detected taking whole image 2K18-CO-243.jpg
Whole image couldn't be taken.2K18-CO-243.jpg is skipped


In [5]:
df = pd.read_excel('dtu/2k18 co students list.xlsx',header=None,sheet_name='Sheet2', names=['Name','Roll_no','photo_name'])

In [12]:
embeddings=[]
iden = []
with open("embeddings.txt","rb") as f:
    embeddings = pickle.load(f)

with open("identity.txt","rb") as f:
    iden = pickle.load(f)
    
model = load_model('face_recog.h5')
    
# define new photo embedding
n = int(input())
for f in os.listdir('./test/'):
    try:
        face_array = extract_face('./test/' + f)
    except:
        print('Face not detected taking whole image {}'.format(f))
        try:
            face_array = extract('./test/' + f)
        except:
            print("Whole image couldn't be taken.{} is skipped".format(f))
            continue
    new_face = asarray(face_array, 'float32')
    new_face = preprocess_input(new_face, version=2)
    new_face = new_face.reshape(1,224,224,3)
    new_user_emb = model.predict(new_face)
    new_user_emb = new_user_emb.reshape(-1)

    # verify known photos of sharon
    ans = []
    for emb in embeddings:
        ans.append(cosine(emb,new_user_emb))
    index = np.argsort(ans)
    for i in range(0,n):
        per = (1-ans[index[i]])*100
        per = "{:.2f}".format(per)
        print('{} Matched with {} with {}%'.format(f, iden[index[i]],per))
        for j,a in enumerate(df['photo_name']):
            if a + '.jpg'== iden[index[i]]:
                print(df.iloc[j,0:2])
        fig, ax = plt.subplots(1,2, figsize=(10, 10))
        pers = pyplot.imread('./test/' + f)
        matc = pyplot.imread('./dataset/' + iden[index[i]])
        ax[0].imshow(pers)
        ax[1].imshow(matc)
        plt.show()
    print('\n')

1
Face not detected taking whole image 20LE1067.png
Whole image couldn't be taken.20LE1067.png is skipped
Face not detected taking whole image 20LE1083.png
Whole image couldn't be taken.20LE1083.png is skipped


In [14]:
iden

['2K18-A1-03.jpg',
 '2K18-A3-280.jpg',
 '2K18-B1-86.jpg',
 '2K18-B1-98.jpg',
 '2K18-B2-229.jpg',
 '2K18-B2-255.jpg',
 '2K18-CO-001.jpg',
 '2K18-CO-002.jpg',
 '2K18-CO-003.jpg',
 '2K18-CO-004.jpg',
 '2K18-CO-005.jpg',
 '2K18-CO-006.jpg',
 '2K18-CO-007.jpg',
 '2K18-CO-008.jpg',
 '2K18-CO-009.jpg',
 '2K18-CO-010.jpg',
 '2K18-CO-011.jpg',
 '2K18-CO-012.jpg',
 '2K18-CO-013.jpg',
 '2K18-CO-014.jpg',
 '2K18-CO-015.jpg',
 '2K18-CO-016.jpg',
 '2K18-CO-017.jpg',
 '2K18-CO-018.jpg',
 '2K18-CO-019.jpg',
 '2K18-CO-020.jpg',
 '2K18-CO-021.jpg',
 '2K18-CO-022.jpg',
 '2K18-CO-023.jpg',
 '2K18-CO-024.jpg',
 '2K18-CO-025.jpg',
 '2K18-CO-026.jpg',
 '2K18-CO-027.jpg',
 '2K18-CO-028.jpg',
 '2K18-CO-029.jpg',
 '2K18-CO-030.jpg',
 '2K18-CO-031.jpg',
 '2K18-CO-032.jpg',
 '2K18-CO-033.jpg',
 '2K18-CO-034.jpg',
 '2K18-CO-035.jpg',
 '2K18-CO-036.jpg',
 '2K18-CO-037.jpg',
 '2K18-CO-038.jpg',
 '2K18-CO-039.jpg',
 '2K18-CO-040.jpg',
 '2K18-CO-041.jpg',
 '2K18-CO-042.jpg',
 '2K18-CO-043.jpg',
 '2K18-CO-044.jpg',
 '2